<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Data_Cleanup_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.5 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=2a9076353f72c23c83f8c28ef805211ba51207c6280f943d68d986e0c0cb222a
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=f686851b364d3afed532e33aa256ae3a212341a7abe09eaea899ed23b0521e1f
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)

In [7]:
mines.head()

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,NaN,TN2404,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,NaN,TN2405,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,NaN,TN2756,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,NaN,TN2497,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,NaN,TN2284023,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."


In [8]:
import pandas as pd
import os

directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['start_date'] = directory_files[z]
    df_file['start_date'] = df_file['start_date'].str.slice(3, 7)
    df_file['start_date'] = df_file['start_date']+'-07-01'
    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df_file['start_date'] = pd.to_datetime(df_file['start_date'])

    df = pd.concat([df, df_file])

    z+=1


In [9]:
df

,ï»¿,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,...,built,bare,snow_and_ice,label,Company,mine_id,orig_perm_id,start_date,State,end_date
0,3,20230718T161839_20230718T162101_T17SLC,-82.962111,38.040957,1689697971141,0.032271,0.742175,0.033115,0.028178,0.029994,...,0.029040,0.036130,0.035817,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023-07-01,ky,NaN
1,9,20230718T161839_20230718T162101_T17SLC,-82.963908,38.042754,1689697971141,0.032551,0.741282,0.033707,0.028543,0.030174,...,0.028126,0.036269,0.035345,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023-07-01,ky,NaN
2,15,20230718T161839_20230718T162101_T17SLC,-82.962111,38.042754,1689697971141,0.031584,0.732819,0.034490,0.028255,0.029593,...,0.030514,0.038793,0.033416,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023-07-01,ky,NaN
3,19,20230705T160829_20230705T162114_T17SLC,-82.960315,38.042754,1688574175051,0.024305,0.746942,0.039173,0.024682,0.038829,...,0.029820,0.027902,0.035887,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023-07-01,ky,NaN
4,21,20230718T161839_20230718T162101_T17SLC,-82.960315,38.042754,1689697971141,0.030797,0.746335,0.035748,0.026772,0.032971,...,0.027994,0.032573,0.033623,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023-07-01,ky,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10081,12,20170726T160859_20170726T161854_T17SMB,-81.955998,37.766073,1501085934130,0.080749,0.049291,0.047412,0.040685,0.050573,...,0.244145,0.306444,0.045043,6.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017-07-01,wv,2017-07-31
10082,13,20170726T160859_20170726T161854_T17SMB,-81.954202,37.766073,1501085934130,0.055003,0.027637,0.028270,0.034314,0.049508,...,0.081053,0.531647,0.058603,7.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017-07-01,wv,2017-07-31
10083,1,20170726T160859_20170726T161854_T17SMC,-81.102599,38.278113,1501085934130,0.024015,0.767977,0.030674,0.022576,0.035489,...,0.028703,0.028759,0.033115,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017-07-01,wv,2017-07-31
10084,1,20170726T160859_20170726T161854_T17SMC,-81.097209,38.272723,1501085934130,0.024376,0.762291,0.030845,0.022416,0.033265,...,0.030121,0.032255,0.030985,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017-07-01,wv,2017-07-31


In [10]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file

0	#419bdf	water
1	#397d49	trees
2	#88b053	grass
3	#7a87c6	flooded_vegetation
4	#e49635	crops
5	#dfc35a	shrub_and_scrub
6	#c4281b	built
7	#a59b8f	bare
8	#b39fe1	snow_and_ice


label0 = df[['orig_perm_id', 'label','mine_id']][df['label']==0.0].groupby(['orig_perm_id', 'label']).count()
label0 = label0.rename(columns={"mine_id": "water"})
label1 = df[['orig_perm_id', 'label','mine_id']][df['label']==1.0].groupby(['orig_perm_id', 'label']).count()
label1 = label1.rename(columns={"mine_id": "trees"})
label2 = df[['orig_perm_id', 'label','mine_id']][df['label']==2.0].groupby(['orig_perm_id', 'label']).count()
label2 = label2.rename(columns={"mine_id": "grass"})
label3 = df[['orig_perm_id', 'label','mine_id']][df['label']==3.0].groupby(['orig_perm_id', 'label']).count()
label3 = label3.rename(columns={"mine_id": "flooded_vegetation"})
label4 = df[['orig_perm_id', 'label','mine_id']][df['label']==4.0].groupby(['orig_perm_id', 'label']).count()
label4 = label4.rename(columns={"mine_id": "crops"})
label5 = df[['orig_perm_id', 'label','mine_id']][df['label']==5.0].groupby(['orig_perm_id', 'label']).count()
label5 = label5.rename(columns={"mine_id": "shrub_and_scrub"})
label6 = df[['orig_perm_id', 'label','mine_id']][df['label']==6.0].groupby(['orig_perm_id', 'label']).count()
label6 = label6.rename(columns={"mine_id": "built"})
label7 = df[['orig_perm_id', 'label','mine_id']][df['label']==7.0].groupby(['orig_perm_id', 'label']).count()
label7 = label7.rename(columns={"mine_id": "bare"})
label8 = df[['orig_perm_id', 'label','mine_id']][df['label']==8.0].groupby(['orig_perm_id', 'label']).count()
label8 = label8.rename(columns={"mine_id": "snow_and_ice"})

df = pd.merge(df, df[['orig_perm_id', 'start_date']].groupby('orig_perm_id').count(), on='orig_perm_id', how='left')
df = df.rename(columns={"start_date_y": "Observations", "start_date_x": "Date"})

df = pd.merge(df[['Company', 'Date', 'Observations', 'orig_perm_id']], label0[['water']], on='orig_perm_id', how='left').drop_duplicates()
df = pd.merge(df, label1[['trees']], on='orig_perm_id', how='left')
df = pd.merge(df, label2[['grass']], on='orig_perm_id', how='left')
df = pd.merge(df, label3[['flooded_vegetation']], on='orig_perm_id', how='left')
df = pd.merge(df, label4[['crops']], on='orig_perm_id', how='left')
df = pd.merge(df, label5[['shrub_and_scrub']], on='orig_perm_id', how='left')
df = pd.merge(df, label6[['built']], on='orig_perm_id', how='left')
df = pd.merge(df, label7[['bare']], on='orig_perm_id', how='left')
df = pd.merge(df, label8[['snow_and_ice']], on='orig_perm_id', how='left')
df.fillna(0, inplace=True)
df

,Company,Date,Observations,orig_perm_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice
0,B & C ENERGY INC,2023-07-01,487,8640180,0.0,463.0,0.0,0.0,9.0,7.0,0.0,8.0,0.0
1,RIDNER COAL CO INC,2023-07-01,8,9180010,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HERBERT WELLS,2023-07-01,8,0320080,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MOUNT VICTORY COAL CO INC,2023-07-01,40,404774X,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVERGREEN MINING INC,2023-07-01,70,8260501,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31562,LO-MING COAL CORP.,2017-07-01,2,U502397,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31563,"APPALACHIAN FUELS, LLC",2017-07-01,144,S304188,0.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31564,"CHICOPEE COAL COMPANY, INC.",2017-07-01,28,S007385,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31565,BELVA COAL COMPANY,2017-07-01,32,S009482,0.0,28.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0


In [13]:
mines

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,NaN,TN2404,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,NaN,TN2405,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,NaN,TN2756,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,NaN,TN2497,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,NaN,TN2284023,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25119,25120,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,5370.984081,9.330482e+05,"MULTIPOLYGON (((-9540804.196 4143151.210, -954..."
25120,25121,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,534.608228,1.452293e+04,"MULTIPOLYGON (((-9541424.447 4142622.814, -954..."
25121,25122,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,507.253927,1.464629e+04,"MULTIPOLYGON (((-9540236.664 4142842.258, -954..."
25122,25123,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,4223.617167,7.008642e+05,"MULTIPOLYGON (((-9540725.233 4143212.597, -954..."


In [16]:
len(df)

31567

In [17]:
len(pd.merge(df, mines,  how='inner', left_on=['orig_perm_id'], right_on = ['permit_id']))

22560

In [18]:
df = pd.merge(df, mines,  how='inner', left_on=['orig_perm_id'], right_on = ['permit_id'])

In [19]:
df

,Company,Date,Observations,orig_perm_id,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
0,RIDNER COAL CO INC,2023-07-01,8,9180010,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,712.474436,18164.357641,"MULTIPOLYGON (((-9366825.467 4427104.386, -936..."
1,RIDNER COAL CO INC,2021-07-01,8,9180010,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,712.474436,18164.357641,"MULTIPOLYGON (((-9366825.467 4427104.386, -936..."
2,RIDNER COAL CO INC,2019-07-01,8,9180010,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,712.474436,18164.357641,"MULTIPOLYGON (((-9366825.467 4427104.386, -936..."
3,HERBERT WELLS,2023-07-01,8,0320080,0.0,6.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,1376.228768,18377.121477,"MULTIPOLYGON (((-9260908.255 4608200.549, -926..."
4,HERBERT WELLS,2021-07-01,8,0320080,0.0,6.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,1376.228768,18377.121477,"MULTIPOLYGON (((-9260908.255 4608200.549, -926..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,"MURRAY AMERICAN ENERGY, INC.",2017-07-01,1,U201111,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,1098.112504,48100.500564,"MULTIPOLYGON (((-8946070.064 4782313.995, -894..."
22556,MINGO LOGAN COAL LLC,2017-07-01,2,U501997,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,2048.174641,55939.170211,"MULTIPOLYGON (((-9109368.083 4567069.474, -910..."
22557,"FARMERS CONSTRUCTION COMPANY, INC.",2017-07-01,1,S005080,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,1624.070339,89226.551981,"MULTIPOLYGON (((-8943187.897 4782217.133, -894..."
22558,"MOUNTAIN EMPIRE LEASING, INC.",2017-07-01,2,S400792,0.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,1.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,8067.189345,407290.401154,"MULTIPOLYGON (((-9081851.554 4485865.243, -908..."
